In [62]:
import numpy as np
import pandas as pd
import data_als
import data_prepro

In [63]:
import torch
import torch.nn as nn
from torch.autograd import Variable 

import torch.optim as optim

In [64]:
train,test = data_prepro.loadData('datasets/data_tr_city.csv','datasets/data_ts_city.csv',trainYearRange1=2019,trainYearRange2=2020)

In [65]:
train = data_prepro.outlierDataToNan(train,low=True,high=False,thres=10.0)
# train = data_prepro.fillZero(train)
train = data_prepro.fillnaBehind(train)
train = data_prepro.fillprevValue(train)


Int64Index([17558, 21595, 25626], dtype='int64')
index                  0
datetime               0
구미 혁신도시배수지 유출유량 적산차    2
year                   0
month                  0
dtype: int64


In [66]:
x_train,y_train = data_prepro.XDataToXAndYSeq(train)

In [67]:
x_test,y_test = data_prepro.XDataToXAndYSeq(test)

In [68]:
x_train.shape, y_train.shape

((17519, 1, 24), (17519, 1))

In [69]:
y_train = y_train.reshape(-1,1,1)

In [70]:
y_train.shape

(17519, 1, 1)

In [71]:
X_train_tensors = Variable(torch.Tensor(x_train))
X_test_tensors = Variable(torch.Tensor(x_test))

y_train_tensors = Variable(torch.Tensor(y_train))
y_test_tensors = Variable(torch.Tensor(y_test))

X_train_tensors_final = torch.reshape(X_train_tensors,   (X_train_tensors.shape[0], X_train_tensors.shape[1], X_train_tensors.shape[2]))
X_test_tensors_final = torch.reshape(X_test_tensors,  (X_test_tensors.shape[0], X_test_tensors.shape[1], X_test_tensors.shape[2])) 

print("Training Shape", X_train_tensors_final.shape, y_train_tensors.shape)
print("Testing Shape", X_test_tensors_final.shape, y_test_tensors.shape) 

Training Shape torch.Size([17519, 1, 24]) torch.Size([17519, 1, 1])
Testing Shape torch.Size([8399, 1, 24]) torch.Size([8399, 1])


In [72]:
# try:
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # device
# except:
#     device = "cpu"
# print(torch.cuda.get_device_name(0))

In [73]:
class LSTM1(nn.Module):
  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTM1, self).__init__()
    self.num_classes = num_classes #number of classes
    self.num_layers = num_layers #number of layers
    self.input_size = input_size #input size
    self.hidden_size = hidden_size #hidden state
    self.seq_length = seq_length #sequence length
 
    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                      num_layers=num_layers, batch_first=True) #lstm
    self.fc_1 =  nn.Linear(hidden_size, 1024) #fully connected 1
    self.fc = nn.Linear(1024, num_classes) #fully connected last layer

    self.relu = nn.ReLU() 
    self.tanh = nn.Tanh()

  def forward(self,x):
    h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to("cpu") #hidden state
    c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to("cpu") #internal state   
    # Propagate input through LSTM

    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
   
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = self.relu(hn)
    out = self.fc_1(out) #first Dense
    out = self.relu(out) #relu
    out = self.fc(out) #Final Output
   
    return out 

In [74]:
num_epochs = 30000 #1000 epochs
learning_rate = 0.00001 #0.001 lr

input_size = 24 #number of features
hidden_size = 256 #number of features in hidden state
num_layers = 3 #number of stacked lstm layers

num_classes = 1 #number of output classes 

In [75]:
lstm1 = LSTM1(num_classes, input_size, hidden_size, num_layers, X_train_tensors_final.shape[1]).to("cpu")

loss_function = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm1.parameters(), lr=learning_rate)  # adam optimizer

In [ ]:
for epoch in range(num_epochs):
    outputs = lstm1.forward(X_train_tensors_final.to("cpu")) #forward pass
    optimizer.zero_grad() #caluclate the gradient, manually setting to 0
    
    # obtain the loss function
    loss = loss_function(outputs, y_train_tensors.to("cpu"))

    loss.backward() #calculates the loss of the loss function
    
    optimizer.step() #improve from loss, i.e backprop
  
    print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

c:\Users\ParkJeongSu\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([17519, 1, 1])) that is different to the input size (torch.Size([52557, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 0, loss: 91310.21094


KeyboardInterrupt: 